# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Шаг 1. Откройте файл с данными и изучите общую информацию

In [20]:
import pandas as pd
data = pd.read_csv('data.csv')
data.info()
data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


**Вывод**

Прочитали файл. Посмотрели его содержание и типы данных. Табоица содержит 21525 записей. Cтолбцы "days_employed" и "total_income" имет меньшее количество значений чем остальные столбцы. Столбец "days_employed" - общий трудовой стаж имеет отрицательные значения. Отрицательного стажа быть не должно. В столбце "education" разный регистр.

## Шаг 2. Предобработка данных

### Обработка пропусков

In [21]:

print('Количество пропусков до замены')
print(data.isna().sum())

   

data_grouped_median = data.groupby('income_type').agg({'days_employed': ['median'],'total_income': ['median']}) #Применим функцию median по столбам к сгрупированым данным.
#print(data_grouped_median['total_income']['median'])
income_type = data['income_type'].unique() #получим список групп

#print(data_grouped_median['days_employed']['median'])
#заменим пропуски на медианное знасение соответсвующей группы.
for inc in income_type:
    data.loc[data['income_type'] == inc,['total_income']] = data.loc[data['income_type'] == inc,['total_income']].fillna(data_grouped_median['total_income']['median'][inc])
    data.loc[data['income_type'] == inc,['days_employed']] = data.loc[data['income_type'] == inc,['days_employed']].fillna(data_grouped_median['days_employed']['median'][inc])
#data['total_income'] = data.loc[data['income_type'] == 'госслужащий',['total_income']].fillna(data_grouped_median['total_income']['median']['госслужащий'])
print('')  
print('Количество пропусков после замены')   
print(data.isna().sum())
#data.loc[data['income_type'] == 'госслужащий',['total_income']]
data.head()



Количество пропусков до замены
children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Количество пропусков после замены
children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


**Вывод**

В столбцах "days_employed" (стаж) и "total_income" (зарплата) имеется одинаковое количество пропущенных значений. Пропуски составлят 10%. Возможно в какойто момент эти данные на записывались. Заменили пропущенные значения на медианные значения для соответсвующей группы income_type чтобы эти замены как можно меньше воияли на общуб сатаистику.

### Замена типа данных

In [22]:
# заменим тип данных  в столбце days_employed на целочисленный
data['days_employed'] = data['days_employed'].astype('int')

print('Максимальный трудовой стаж {:.0f}, лет'.format(data['days_employed'].max()/365))
print('Минимальный трудовой стаж {:.0f}, лет'.format(data['days_employed'].min()/365))
print('')
print('Медианный трудовой стаж по группам, лет')
print((data.groupby('income_type')['days_employed'].median() / 365).astype('int'))






Максимальный трудовой стаж 1101, лет
Минимальный трудовой стаж -50, лет

Медианный трудовой стаж по группам, лет
income_type
безработный        1003
в декрете            -9
госслужащий          -7
компаньон            -4
пенсионер          1000
предприниматель      -1
сотрудник            -4
студент              -1
Name: days_employed, dtype: int32


In [23]:
data['total_income'] = data['total_income'].astype('int') # заменим тип данных  в столбце total_income на целочисленный
print('Максимальный ежемесячный доход {:.0f}, руб'.format(data['total_income'].max()))
print('Минимальный ежемесячный доход {:.0f}, руб'.format(data['total_income'].min()))
print('')
print('Медианный ежемесячный доход по группам, руб')
print((data.groupby('income_type')['total_income'].median()).astype('int'))

Максимальный ежемесячный доход 2265604, руб
Минимальный ежемесячный доход 20667, руб

Медианный ежемесячный доход по группам, руб
income_type
безработный        131339
в декрете           53829
госслужащий        150447
компаньон          172357
пенсионер          118514
предприниматель    499163
сотрудник          142594
студент             98201
Name: total_income, dtype: int32


**Вывод**

В столбцах "days_employed" и "total_income" поменяли тип данных с float на int. Не все данные в столюце "days_employed" соответсвуют заявленным. Наблюдаются аномально высокие значения для пенсионеров и безработных. С зарплатми вроде все нормально, если это Москва.

### Обработка дубликатов

In [24]:
data['education'] = data['education'].str.lower()
print('Колличество дубликатов',data.duplicated().sum())
data = data.drop_duplicates().reset_index(drop = True)
data.duplicated().sum()

Колличество дубликатов 71


0

**Вывод**

Обнаружено 71 дубликат. Все они дропнуты. Для поиска и устранения дубликатов использовали метод drop_duplicates() со сбросом индексов. Чтобы исключить повторения предварительно привели все строковые параметры к нижнему регистру.

### Лемматизация

In [7]:
from pymystem3 import Mystem
m = Mystem()
lemmas = []
purpose_cat = []
# запишем в массив lemmas леммы за столбца purpose
for row in range(len(data)):
    lem = m.lemmatize(data.loc[row]['purpose'])
    for l in lem:
        lemmas.append(l)
        
# создадим dataframe чтобы посчитать колличество уникальных значений применив метод value_counts
col = ['lemmas']
lemmas_col = pd.DataFrame(data = lemmas, columns = col)
lemmas_col.info()
print(lemmas_col['lemmas'].value_counts()) 
#m.lemmatize(data.loc[0]['purpose'])
#data.head()





<class 'pandas.core.frame.DataFrame'>
RangeIndex: 110048 entries, 0 to 110047
Data columns (total 1 columns):
lemmas    110048 non-null object
dtypes: object(1)
memory usage: 859.9+ KB
                  33570
\n                21454
недвижимость       6351
покупка            5897
жилье              4460
автомобиль         4306
образование        4013
с                  2918
операция           2604
свадьба            2324
свой               2230
на                 2222
строительство      1878
высокий            1374
получение          1314
коммерческий       1311
для                1289
жилой              1230
сделка              941
дополнительный      906
заниматься          904
проведение          768
сыграть             765
сдача               651
семья               638
собственный         635
со                  627
ремонт              607
подержанный         486
подержать           478
приобретение        461
профильный          436
Name: lemmas, dtype: int64


**Вывод**

Лемматизировали цели получения кредита. Среди целей кредита можно воделить следующие: недвижимость, жилье, автомобиль, образование, свадьба, ремонт.

### Категоризация данных

In [8]:
# запишем массив с категориями
purpose_cat = []
# для каждого клиента выделим из данных лемматизации категорию и запишем ее.
for row in range(len(data)):
    lem = m.lemmatize(data.loc[row]['purpose'])
    
    if 'недвижимость' in lem:
        purpose_cat.append('жилье')
    elif 'жилье' in lem:
        purpose_cat.append('жилье')
    elif 'автомобиль' in lem:
        purpose_cat.append('автомобиль')
    elif 'образование' in lem:
        purpose_cat.append('образование')
    elif 'свадьба' in lem:
        purpose_cat.append('свадьба')
    elif 'ремонт' in lem:
        purpose_cat.append('ремонт')
    else:
        purpose_cat.append('-')
        
# создадим dataframe чтобы посчитать колличество уникальных значений применив метод value_counts    
col = ['purpose_cat']
purpose_cat_col = pd.DataFrame(data = purpose_cat, columns = col)
print('')
print('Распределение по категориям целей кредита, чел')
print(purpose_cat_col['purpose_cat'].value_counts())
# добавим столбец с категориями в общую таблицу
data['purpose_cat'] = purpose_cat

total_income_cat = []
# для каждого клиента определим уровень дохода и отнесем его к группе.
for row in range(len(data)):
    ti = data.loc[row]['total_income']
    if ti <= 100000:
        total_income_cat.append('бедный')
    if 100000 < ti <= 500000:
        total_income_cat.append('средний')
    if 500000 < ti:
        total_income_cat.append('богатый')
                
col = ['total_income_cat']
total_income_cat_col = pd.DataFrame(data = total_income_cat, columns = col)
print('')
print('Распределение по категориям дохода, чел')
print(total_income_cat_col['total_income_cat'].value_counts())
# добавим столбец с категориями в общую таблицу
data['total_income_cat'] = total_income_cat


data.head()



Распределение по категориям целей кредита, чел
жилье          10811
автомобиль      4306
образование     4013
свадьба         2324
Name: purpose_cat, dtype: int64

Распределение по категориям дохода, чел
средний    16769
бедный      4463
богатый      222
Name: total_income_cat, dtype: int64


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_cat,total_income_cat
0,1,-8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,жилье,средний
1,1,-4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,автомобиль,средний
2,0,-5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,жилье,средний
3,3,-4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,образование,средний
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,свадьба,средний


**Вывод**

Разложили цели кредита на выделенные выше категории. Категория "ремонт" как отдельня не выделилась. Видимо она попала в категорию жилье.

## Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

**Вывод**

In [9]:
print('Распредеоение клиентов по колличеству детей:')
print(data.groupby('children')['debt'].count())
print('')
print('Процент неплательщиков в зависимости от колличества детей, %:')
#сгруппируем данные по столбцу children и посчитаем отношение дожников к общему колличеству человек в группе и выразим в процентах
(data.groupby('children')['debt'].sum() / data.groupby('children')['debt'].count() * 100).astype('int') 

Распредеоение клиентов по колличеству детей:
children
-1        47
 0     14091
 1      4808
 2      2052
 3       330
 4        41
 5         9
 20       76
Name: debt, dtype: int64

Процент неплательщиков в зависимати от колличества детей, %:


children
-1      2
 0      7
 1      9
 2      9
 3      8
 4      9
 5      0
 20    10
Name: debt, dtype: int64

Ярко выраженной зависимоти не наблюдается. Результаты в пределах 7-9% Отбнаружилось 2 артефакта, которые в выводах учитывать не будем. Отрицательных детей быть не может. Возможно опечатка. 20 детей  слишком много и таких людей не может быть так много. Скорей всего это ошибка. Также выделяется группа с 5ю детьми, но она слишком малочисленна чтобы делать по ней выводы. Её можно обьединить с соседней группой.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [15]:
print('Распределение клиентов по семейному положению, чел:')
print(data.groupby('family_status')['debt'].count())
print('')
print('Процент неплательщиков в зависимости от семейного положения, %:')
#сгруппируем данные по столбцу family_status и посчитаем отношение дожников к общему колличеству человек в группе и выразим в процентах
(data.groupby('family_status')['debt'].sum() / data.groupby('family_status')['debt'].count() * 100).astype('int')

Распределение клиентов по семейному положению, чел:
family_status
Не женат / не замужем     2810
в разводе                 1195
вдовец / вдова             959
гражданский брак          4151
женат / замужем          12339
Name: debt, dtype: int64

Процент неплательщиков в зависимости от семейного положения, %:


family_status
Не женат / не замужем    9
в разводе                7
вдовец / вдова           6
гражданский брак         9
женат / замужем          7
Name: debt, dtype: int64

**Вывод**

Ярко выраженной зависимоти не наблюдается. Результаты в пределах 6-9%

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [14]:
print('Распределение клиентов по уровню ежемесячного дохода, чел:')
print(data.groupby('total_income_cat')['debt'].count())
print('')
print('Процент неплательщиков в зависимати от ежемесячного дохода, %:')
#сгруппируем данные по столбцу total_income_cat и посчитаем отношение дожников к общему колличеству человек в группе и выразим в процентах
(data.groupby('total_income_cat')['debt'].sum() / data.groupby('total_income_cat')['debt'].count() * 100).astype('int')

Распределение клиентов по уровню ежемесячного дохода, чел:
total_income_cat
бедный      4463
богатый      222
средний    16769
Name: debt, dtype: int64

Процент неплательщиков в зависимати от ежемесячного дохода, %:


total_income_cat
бедный     7
богатый    6
средний    8
Name: debt, dtype: int64

**Вывод**

Ярко выраженной зависимоти не наблюдается. Результаты в пределах 6-8%

- Как разные цели кредита влияют на его возврат в срок?

In [13]:
print('Распредеоение клиентов в зависимости от целей кредита, чел:')
print(data.groupby('purpose_cat')['debt'].count())
print('')
print('Процент неплательщиков в зависимости от целей кредита, %:')
#сгруппируем данные по столбцу purpose_cat и посчитаем отношение дожников к общему колличеству человек в группе и выразим в процентах
(data.groupby('purpose_cat')['debt'].sum() / data.groupby('purpose_cat')['debt'].count() * 100).astype('int')

Распредеоение клиентов в зависимости от целей кредита, чел:
purpose_cat
автомобиль      4306
жилье          10811
образование     4013
свадьба         2324
Name: debt, dtype: int64

Процент неплательщиков в зависимости от целей кредита, %:


purpose_cat
автомобиль     9
жилье          7
образование    9
свадьба        8
Name: debt, dtype: int64

**Вывод**

Ярко выраженной зависимоти не наблюдается. Результаты в пределах 7-9%

## Шаг 4. Общий вывод

Правнализировали статистику платежеспособности клиентов банка. Не нашли никакой интересной иформации. Процент  должников существенно не зависит не от семейного положения, ни от уровня дохода, ни от колличества детей, ни от целей крелита. Нужно выдавать кредит всем категорям граждан без дискриминации. Если смотреть по наилучшим показателям то идеальный земщик это богатый вдовец без детей берущий кредит на покупрку жилья.
Надо разобратся со сбором статистики по общему трудовому стажу. Данные явно записываются не правильно. 